<a href="https://colab.research.google.com/github/nanditavenkatesh/-Fine-Tuning-FLAN-T5-for-Python-Programming-Assistance/blob/main/Fine_tuning_FLAN_T5_for_Python_Question_Answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

Sources:<br>
https://www.datacamp.com/tutorial/flan-t5-tutorial<br>
https://www.markhneedham.com/blog/2023/06/19/huggingface-max-length-generation-length-deprecated/<br>
https://stackoverflow.com/questions/66091226/runtimeerror-expected-all-tensors-to-be-on-the-same-device-but-found-at-least<br>
https://huggingface.co/docs/transformers/en/main_classes/trainer#transformers.Seq2SeqTrainer<br>

In [ ]:
%%bash
pip install nltk
pip install datasets
pip install transformers[torch]
pip install tokenizers
pip install evaluate
pip install rouge_score
pip install sentencepiece
pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 7.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-no

In [ ]:
# !pip install langdetect
import pandas as pd
from google.colab import drive
from bs4 import BeautifulSoup

import nltk
nltk.download("punkt", quiet=True)

import evaluate
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

import datasets
import pandas as pd
from sklearn.model_selection import train_test_split

from textwrap import fill
import torch

In [ ]:
#@title Mount the Google drive.
google_drive = "/drive/" #@param {type:"string"}
drive.mount(google_drive, force_remount=True)

Mounted at /drive/


#Data Processing

In [ ]:
#@title Initialize paths and common variables
first_time = False #@param {type: "boolean"}
already_have_preprocessed = True #@param {type:"boolean"}
path = '/drive/MyDrive/NLU Final Project/'
answer_path = path + 'Stackoverflow_python/Answers.csv'
question_path = path + 'Stackoverflow_python/Questions.csv'
tag_path = path + 'Stackoverflow_python/Tags.csv'
filtered_file_path = path + 'Stackoverflow_python/filtered_df.csv'

In [ ]:
#@title Read from CSV
if not already_have_preprocessed:
  f = open(question_path,encoding="utf-8", errors='ignore')
  df_questions = pd.read_csv(f)
  f.close()
  len(df_questions)

  f = open(answer_path,encoding="utf-8", errors='ignore')
  df_answers = pd.read_csv(f)
  f.close()
  len(df_answers)

  df_tags = pd.read_csv(tag_path)
  len(df_tags)

In [ ]:
#@title Preprocessing train and test df
if first_time:
  def remove_html_tags(text):
      soup = BeautifulSoup(text, "html.parser")
      cleaned_text = soup.get_text(separator=" ")
      cleaned_text = cleaned_text.replace("\n", " ")
      return cleaned_text.strip()

  def preprocess(title, body):
      # Remove HTML tags
      result = "Title: " + remove_html_tags(title) + "Body: " + remove_html_tags(body)
      return result

  def has_non_utf(text):
    if any(ord(char) >= 128 for char in text):
      return None
    return text

  if not already_have_preprocessed:
    #Processing the questions
    df_questions['Title_Question'] = df_questions.apply(lambda x: preprocess(x['Title'], x['Body']), axis=1)
    #Processing the answers
    df_answers['Cleaned_Answer'] = df_answers['Body'].apply(remove_html_tags)
    merged_df = pd.merge(df_answers, df_questions[['Id', 'Title_Question']], left_on='ParentId', right_on='Id', how='left')

    df = merged_df[['ParentId','Title_Question', 'Cleaned_Answer','Score']]
    df.columns = ['Id','Input','Label','Score']
    df = df[df['Score'] >= 2]
    tempdf = df.groupby('Id', group_keys=False).apply(lambda x: x.nlargest(2, 'Score')).reset_index(drop=True)
    df = tempdf.sample(frac=1.0, random_state=42).reset_index(drop=True)
    df = df[['Input', 'Label']]

    df['Filtered Input'] = df['Input'].apply(lambda x: has_non_utf(x))
    df['Filtered Label'] = df['Label'].apply(lambda x: has_non_utf(x))

    #Creating a filtered_df with just UTF-8 characters
    filtered_df = df[df['Filtered Input'].notnull()]
    filtered_df = filtered_df[filtered_df['Filtered Label'].notnull()]
    filtered_df = filtered_df.drop(['Filtered Input', 'Filtered Label'], axis=1)

    filtered_df.to_csv(filtered_file_path, index=False, escapechar='\\')
  else:
    filtered_df = pd.read_csv(filtered_file_path)

  train_df, test_df = train_test_split(filtered_df, test_size=0.2)
  train_df.to_csv(path + 'Stackoverflow_python/train_df.csv', index=False)
  test_df.to_csv(path + 'Stackoverflow_python/test_df.csv', index=False)

else:
  train_df = pd.read_csv(path + 'Stackoverflow_python/train_df.csv')
  test_df = pd.read_csv(path + 'Stackoverflow_python/test_df.csv')

In [ ]:
counter_file_path = path + 'Stackoverflow_python/training_counter.txt'
def read_counter():
    try:
        with open(counter_file_path, 'r') as file:
            count = int(file.read().strip())
    except FileNotFoundError:
        # Initialize counter if the file doesn't exist
        count = 0
        write_counter(count)
    return count

def write_counter(count):
    with open(counter_file_path, 'w') as file:
        file.write(str(count))

current_count = read_counter()

print(f"Current count: {current_count}")

Current count: 2


###Creating train and test set

In [ ]:
#@title Convert to dataset object
# dataset = datasets.Dataset.from_pandas(filtered_df)
train_size = 20000 #@param {type: "integer"}
test_size = 2000 #@param {type: "integer"}

train_dataset = datasets.Dataset.from_pandas(train_df[current_count * train_size: min(current_count *train_size + train_size, len(train_df))])
test_dataset = datasets.Dataset.from_pandas(test_df[current_count * test_size: min(current_count * test_size + test_size, len(test_df))])

In [ ]:
train_dataset_dict = train_dataset.train_test_split(test_size=0.05)

#Initialize model

In [ ]:
not_finetuned = False #@param {type: "boolean"}
if not_finetuned:
  # Load the tokenizer, model, and data collator
  MODEL_NAME = "google/flan-t5-base"

  tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
  model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
  data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
else:
  checkpoint = "checkpoint-14000" #@param {type: "string"}
  checkpoint_path = path + 'Save Results/' + checkpoint

  finetuned_model = T5ForConditionalGeneration.from_pretrained(checkpoint_path)
  tokenizer = T5Tokenizer.from_pretrained(checkpoint_path)
  data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=finetuned_model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# We prefix our tasks with "answer the question"
prefix = "Please answer this question: "

# Define the preprocessing function
def preprocess_function(examples):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   # The "inputs" are the tokenized answer:
   inputs = [prefix + doc for doc in examples["Input"]]
   model_inputs = tokenizer(inputs, max_length=300, truncation=True, padding='max_length')

   # The "labels" are the tokenized outputs:
   labels = tokenizer(text_target=examples["Label"],
                      max_length=300,
                      truncation=True, padding='max_length')

   model_inputs["labels"] = labels["input_ids"]
   return model_inputs

In [ ]:
tokenized_train = train_dataset_dict.map(preprocess_function, batched=True, batch_size=32)
tokenized_test = test_dataset.map(preprocess_function, batched=True, batch_size=32)

Map:   0%|          | 0/19000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

##Evaluation metrics

In [ ]:
#@title Rouge evaluation
metric = evaluate.load("rouge")
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # print(preds)
    # decode preds and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # decoded_preds = tokenizer.decode(preds)
    # decoded_labels = tokenizer.decode(labels)

    # rougeLSum expects newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    return result

In [ ]:
#@title Human evaluation
questions = [
    "What is the difference between list and tuple?",
    "How to create a new text file in Python?",
    "How to install Numpy?",
    "How to add 2 lists together?",
    "How to remove a specific element from a list in Python?",
    "How to find the index of an element in a list in Python?",
]

def human_eval(questions, model, tokenizer):
  device = "cuda" if torch.cuda.is_available() else "cpu"
  model.to(device)

  for question in questions:
      inputs = "Please answer this question: " + question

      inputs = tokenizer(inputs, return_tensors="pt")
      inputs = {k: v.to(device) for k, v in inputs.items()}

      outputs = model.generate(**inputs, max_new_tokens=300)
      answer = tokenizer.decode(outputs[0])

      print("\nQuestion:", question)
      print(fill(answer, width=150))

##Trainer

In [ ]:
#@title Training arguments
# Global Parameters
L_RATE = 3e-4
BATCH_SIZE = 4
PER_DEVICE_EVAL_BATCH = 1
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 1
NUM_EPOCHS = 3

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
   output_dir=path + "/Save Results",
   evaluation_strategy="epoch",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
   push_to_hub=False,
   generation_max_length=256,
  #  load_best_model_at_end=True,
)

In [ ]:
#@title Initialize trainer
if not_finetuned:
  trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train["train"],
    eval_dataset=tokenized_train["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
  )
else:
  trainer = Seq2SeqTrainer(
    model=finetuned_model,
    args=training_args,
    train_dataset=tokenized_train["train"],
    eval_dataset=tokenized_train["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
  )

#First 20000 samples

###Test model before training

In [ ]:
trainer.predict(tokenized_test)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


PredictionOutput(predictions=array([[    0,     3,    75, ...,     0,     0,     0],
       [    0,  1044,     5, ..., 11116,  1640,    61],
       [    0,     3,    99, ...,    99,     3,    31],
       ...,
       [    0, 20737,     1, ...,     0,     0,     0],
       [    0,     3,     9, ...,    31,   210,   210],
       [    0,     3,  6667, ...,     1,     0,     0]]), label_ids=array([[   37,     3,   102, ...,     0,     0,     0],
       [18792,    28,  1681, ...,    23,  3524,     1],
       [   27,   133,   653, ...,     0,     0,     0],
       ...,
       [20731,    65,  1192, ...,     0,     0,     0],
       [20737,    31,     7, ...,     0,     0,     0],
       [    3, 25038,    25, ...,     0,     0,     0]]), metrics={'test_loss': 35.453800201416016, 'test_rouge1': 0.0525120624653424, 'test_rouge2': 0.013971405879526238, 'test_rougeL': 0.04647964244109935, 'test_rougeLsum': 0.04838561305019698, 'test_runtime': 1022.3317, 'test_samples_per_second': 2.934, 'test_steps

In [ ]:
human_eval(questions, model, tokenizer)


Question: What is the difference between list and tuple?
<pad> list of tuples</s>

Question: How to create a new text file in Python?
<pad>ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+
ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+
ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+
ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+ctrl+c

Question: How to install Numpy?
<pad> apk</s>

Question: How to add 2 lists together?
<pad> Add the first list to the second list.</s>

Question: How to remove a specific element from a list in Python?
<pad> if the element is not in a list, you can remove it by removing it from the list.</s>

Question: How to find the index of an element in a

###Train/Finetune

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,0.949000,0.858384,0.128639,0.024394,0.101321,0.112149
2,0.845900,0.830757,0.132285,0.028386,0.104900,0.115730
3,0.793000,0.823194,0.129957,0.028096,0.103536,0.114265


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=12750, training_loss=0.8952044115253524, metrics={'train_runtime': 13418.6329, 'train_samples_per_second': 3.801, 'train_steps_per_second': 0.95, 'total_flos': 3.4922624974848e+16, 'train_loss': 0.8952044115253524, 'epoch': 3.0})

###Test after training

In [ ]:
trainer.predict(tokenized_test)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


PredictionOutput(predictions=array([[   0,  148,   54, ...,    8, 4735,    5],
       [   0,  148,  174, ..., 3870,    5,   17],
       [   0,  148,   54, ..., 1748,  599,   52],
       ...,
       [   0,  148,   54, ...,  284, 1368,   16],
       [   0,  148,   54, ...,    9,  115,   75],
       [   0,  148,   54, ...,    7, 4830,    3]]), label_ids=array([[   37,     3,   102, ...,     0,     0,     0],
       [18792,    28,  1681, ...,    23,  3524,     1],
       [   27,   133,   653, ...,     0,     0,     0],
       ...,
       [20731,    65,  1192, ...,     0,     0,     0],
       [20737,    31,     7, ...,     0,     0,     0],
       [    3, 25038,    25, ...,     0,     0,     0]]), metrics={'test_loss': 0.8326656818389893, 'test_rouge1': 0.12872022658262555, 'test_rouge2': 0.027800673045522507, 'test_rougeL': 0.1026371043273614, 'test_rougeLsum': 0.11302472577963663, 'test_runtime': 1375.4494, 'test_samples_per_second': 2.181, 'test_steps_per_second': 2.181})

In [ ]:
human_eval(questions, finetuned_model, tokenizer)


Question: What is the difference between list and tuple?
<pad>Lists are a list of lists. They are a type of tuple. They are a type of tuple. They are a type of tuple. They are a type of tuple. They are a
type of tuple. They are a type of tuple. They are a type of tuple. They are a type of tuple. They are a type of tuple. They are a type of tuple. They
are a type of tuple. They are a type of tuple. They are a type of tuple. They are a type of tuple. They are a type of tuple. They are a type of tuple.
They are a type of tuple. They are a type of tuple. They are a type of tuple. They are a type of tuple. They are a type of tuple. They are a type of
tuple. They are a type of tuple. They are a type of tuple. They are a type of tuple. They are a type of tuple. They are a type of tuple. They are a
type of tuple. They are a type of tuple. They are a type of tuple. They are a type of tuple. They are a type of tuple. They are a type of tuple. They
are a type of tuple. They are a type of tuple. 

In [ ]:
current_count += 1
write_counter(current_count)

#Second 20000 samples

###Train/Finetune

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,1.331300,1.235729,0.177930,0.057260,0.141253,0.152557


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,1.331300,1.235729,0.177930,0.057260,0.141253,0.152557
2,1.248900,1.211209,0.193803,0.063188,0.151754,0.165221


###Test after training

In [ ]:
trainer.predict(tokenized_test)

PredictionOutput(predictions=array([[    0, 20737,  1843, ...,   720,   315,     5],
       [    0,   148,   174, ...,  -100,  -100,  -100],
       [    0,   148,    54, ...,    17,    38,  4752],
       ...,
       [    0,   148,    54, ...,    61,     3,    89],
       [    0,    37,  1053, ...,   356,    19,   990],
       [    0,   148,    54, ...,  -100,  -100,  -100]]), label_ids=array([[18562,    53,   225, ...,     0,     0,     0],
       [  148,    33,  3586, ...,     0,     0,     0],
       [  282,    25,    31, ...,     7,  2423,     1],
       ...,
       [    3,     7,    17, ...,     0,     0,     0],
       [  148,    33,     3, ...,     0,     0,     0],
       [    3, 19832,  2384, ...,  9899,    32,     1]]), metrics={'test_loss': 1.1867262125015259, 'test_rouge1': 0.20348699399166603, 'test_rouge2': 0.06452706109283138, 'test_rougeL': 0.15733348580597084, 'test_rougeLsum': 0.17192710083284685, 'test_runtime': 5443.369, 'test_samples_per_second': 0.367, 'test_steps_

In [ ]:
human_eval(questions, finetuned_model, tokenizer)


Question: What is the difference between list and tuple?
<pad> Lists are a type of list, and tuples are a type of tuple. Lists are a type of list, and tuples are a type of tuple. Lists are a type of list,
and tuples are a type of tuple.</s>

Question: How to create a new text file in Python?
<pad> You can use os.path.join to create a new file. import os os.path.join(os.path.join(os.path.dirname(filename), '.'))</s>

Question: How to install Numpy?
<pad> You can download the numpy package from here: http://www.np.np.ac.uk/downloads/np.np.py</s>

Question: How to add 2 lists together?
<pad> You can use zip() to combine the two lists together: >>> lst = [1,2,3] >>> lst.append([1,2,3]) >>> lst [1, 2, 3]</s>

Question: How to remove a specific element from a list in Python?
<pad> You can use a list comprehension: >>> l = [1,2,3] >>> l.remove('a') [1, 2, 3]</s>

Question: How to find the index of an element in a list in Python?
<pad> You can use the id() function to find the index of the el

In [ ]:
current_count += 1
write_counter(current_count)

#Third 20000 samples

###Train/Finetune

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,1.280100,1.177442,0.187443,0.059865,0.148825,0.159279
2,1.165100,1.156734,0.191937,0.061978,0.152103,0.163683
3,1.119900,1.153487,0.204653,0.068116,0.161000,0.174130


TrainOutput(global_step=14250, training_loss=1.197973553573876, metrics={'train_runtime': 14819.1915, 'train_samples_per_second': 3.846, 'train_steps_per_second': 0.962, 'total_flos': 2.28698256384e+16, 'train_loss': 1.197973553573876, 'epoch': 3.0})

###Test after training

In [ ]:
trainer.predict(tokenized_test)

PredictionOutput(predictions=array([[   0,  148,   54, ..., -100, -100, -100],
       [   0,  148,   54, ..., 1982,    5, 2962],
       [   0,  148,   54, ..., 1042,    5,  148],
       ...,
       [   0,  148,  174, ..., -100, -100, -100],
       [   0,  148,  174, ..., -100, -100, -100],
       [   0,  148,   54, ...,    7,    5,   51]]), label_ids=array([[ 432,   25,  174, ...,    0,    0,    0],
       [  96,  532,  381, ...,    0,    0,    0],
       [1626, 9525,  751, ..., 1218, 2594,    1],
       ...,
       [  86, 8389,   19, ..., 9400, 2423,    1],
       [   3,  157, 2910, ...,    0,    0,    0],
       [ 148,   54,  474, ...,    5,   26,    1]]), metrics={'test_loss': 1.1789946556091309, 'test_rouge1': 0.2004616707625099, 'test_rouge2': 0.06605996745040851, 'test_rougeL': 0.15708727662688343, 'test_rougeLsum': 0.17095621021480722, 'test_runtime': 5386.2864, 'test_samples_per_second': 0.371, 'test_steps_per_second': 0.371})

In [ ]:
human_eval(questions, finetuned_model, tokenizer)


Question: What is the difference between list and tuple?
<pad> Lists are a common way to access a list or tuple. Lists are a common way to access a list or tuple. Lists are a common way to access a list or
tuple. Lists are a common way to access a list or tuple. Lists are a common way to access a list or tuple. Lists are a common way to access a list or
tuple. Lists are a common way to access a list or tuple. Lists are a common way to access a list or tuple. Lists are a common way to access a list or
tuple. Lists are a common way to access a list or tuple. Lists are a common way to access a list or tuple. Lists are a common way to access a list or
tuple. Lists are a common way to access a list or tuple. Lists are a common way to access a list or tuple. Lists are a common way to access a list or
tuple. Lists are a common way to access a list or tuple. Lists are a common way</s>

Question: How to create a new text file in Python?
<pad> You can use os.path.join() to create a new file. im

In [ ]:
current_count

2

In [ ]:
current_count += 1
write_counter(current_count)